In [1]:
from IPython.display import display, Math, Latex

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Defining Linear regression class

In [7]:
class LinReg(object):
    '''
    Linear regression model
    
    y = X@w
    X: A feature matrix
    w: weight vector
    y: label vector
    '''
    
    def __init__(self):
        self.t0 = 200
        self.t1 = 100000
        
    def predict(self, X:np.ndarray) -> np.ndarray:
        '''
        prediction of output label for a given input.
        
        Args:
            X: Feature matrix for given inputs.
            
        Return:
            y: Output label vector as predicted by the given model.
            
            
        '''
        y = X @ self.w
        return y
    
    
    def loss(self, X:np.ndarray, y:np.ndarray) -> float:
        
        '''
        Calculate loss for model based on known labels.
        
        Args: 
            X:Feature matrix for given data
            y: Output label vector as predicted by the given model.
            
        Return:
            Loss
            
        '''
        e = y - self.predict(X)
        return (1/2) * (np.transpose(e) @ e)
    
    
    def rmse(self, X:np.ndarray, y:np.ndarray) -> float:
        
        '''
        Calculate root mean squared error of prediction w.r.t. actual label.
        
        Args:
            X: Feature matrix for given inputs.
            y: Output label vector as predicted by the given model.
        
        Returns:
            Loss
        '''
        
        return np.sqrt((2/X.shape[0]) * self.loss(X,y))
    
    
    
    def fit(self, X:np.ndarray, y:np.ndarray) -> np.ndarray:
        '''Estimates parameters of the linear regression model with normal equation.
        
        Args:
            X: Feature matrix for given inputs.
            y: Output label vector as predicted by the given model.
            
        Returns:
            Weight vector
        '''
        
        self.w = np.linalg.pinv(X) @ y
        return self.w
    
    
    def calculate_gradient(self, X:np.ndarray, y:np.ndarray) -> np.ndarray:
        '''Calculates gradients of loss function w.r.t weight vector on training set.
        
        Arguments:
            X: Feature matrix for training data.
            y: Label vector for training data.
            
        Returns:
            A vector of gradients.
        '''
        
        return np.transose(X) @ (self.predict(X) - y)
    
    
    def update_weights(self, grad: np.ndarray, lr:float) -> np.ndarray:
        '''Updates the weights based on the gradient of loss function.
        
        Weight updates are carried out with the following formula:
            w_new := w_old - lr * grad
        
        Args:
            grad: gradient of loss w.r.t w
            lr: learning rate
        '''
        
        return (self.w - lr*grad)
    
    
    def learning_schedule(self, t):
        return self.t0 / (t + self.t1)
    
    
    def gd(self, X:np.ndarray, y:np.ndarray, num_epochs: int, lr:float) -> np.ndarray:
        '''Estimates parameters of linear regression model through gradient descent.
        
        Args:
            X: Feature matrix for training data.
            y: Label vector for training data.
            
        Returns:
            Weight vector: Final weight vector
            
        '''
        self.w = np.zeros((X.shape[1]))
        self.w_all = []
        self.err_all = []
        for i in np.arange(0,num_epochs):
            dJdW = self.calculate_gradient(X,y)
            self.w_all.append(self.w)
            self.err_all.append(self.loss(X,y))
            self.w = self.update_weights(dJdW, lr)
        return self.w
    
    
    def mbgd(self, X:np.ndarray, y:np.ndarray,
            num_epochs: int, batch_size: int) -> np.ndarray:
        '''Estimates parameters of linear regression model through gradient descent.
        
        Args:
            X: Feature matrix for training data.
            y: Label vector for training data.
            num_epochs: Number of training steps
            batch_size: Number of examples in a batch
        '''
        
        self.w = np.zeros((X.shape[1]))
        self.w_all = []
        self.err_all = []
        mini_batch_id = 0
        
        for epoch in range(num_epochs):
            shuffled_indices = np.random.permutation(X.shape[0])
            X_shuffled = X[shuffled_indices]
            y_shuffled = y[shuffled_indices]
            
            for i in range(0, X.shape[0], batch_size):
                mini_batch_id += 1
                xi = X_shuffled[i:i+batch_size]
                yi = y_shuffled[i:i+batch_size]
                
                self.w_all.append(self.w)
                self.err_all.append(self.loss(xi, yi))
                
                dJdW = 2/batch_size * self.calculate_gradient(xi, yi)
                self.w = self.update_weights(dJdW, self.learning_schedule(mini_batch_id))
        return self.w
    
    
    def sgd(self, X:np.ndarray, y:np.ndarray, num_epochs: int) -> np.ndarray:
        ''' Estimates parameters of linear regression model through gradient descent.
        
        Args:
            X: Feature matrix for training data
            y: Label vector for training data.
            n_epochs: Number of training steps
            batch_size = Number of examples in a batch
            
        Returns: 
            Weight vector: Final weight vector
        '''
        
        # Parameter vector initialized to [0,0]
        self.w = np.zeros((X.shape[1]))
        self.w_all = []
        self.err_all = []
        
        for epoch in range(num_epochs):
            for i in range(X.shape[0]):
                random_index = np.random.randint(X.shape[0])
                xi = X[random_index:random_index+1]
                yi = y[random_index:random_index+1]

                self.w_all.append(self.w)
                self.err_all.append(self.loss(xi, yi))

                gradients = 2 * self.calculate_gradient(xi, yi)
                lr = self.learning_schedule(epoch * X.shape[0] + i)
                self.w = self.update_weights(gradient, lr)

        return self.w